In [1]:
from pathlib import Path
import os
cwd = Path().resolve().parents[1]
activities_path = os.path.join(cwd , 'automagica\\activities.py')
activities_rst_path = os.path.join(cwd , 'docs\\source\\activities.rst')
github_readme_path = os.path.join(cwd , 'readme.md')

In [2]:
from pprint import pprint

with open(activities_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

categories = []

for i, line in enumerate(lines):
    if line == '"""\n': # Category
        if lines[i+1].strip(): # Not empty
            name = lines[i+1].strip()
            icon = lines[i+2].strip().replace('Icon: ','')
            if not icon:
                raise Exception
            category = {'name': name, 'icon': icon, 'activities':[]}
            categories.append(category)
            
    if line.startswith('def ') or line.startswith('    def '): # Function definition
        if lines[i-1].strip() == '@activity':
            activity = {}
            if '__init__' in line:
                if '(' in lines[i-2]:
                    parsed_class_line = lines[i-2].split('(')[0] + ':'
                else:
                    parsed_class_line = lines[i-2]
                arguments = line.split('(')[-1].replace('):','')
                func_call_raw = (parsed_class_line.replace('class ','').replace(':', '(') + arguments + ')').replace('\n','')
                activity['function_call'] = func_call_raw.replace('self, ','')

            else:
                func_call_raw = line.replace('def ','').replace(':', '').strip()
                activity['function_call'] = func_call_raw.replace('self, ','')
            activity['name'] = lines[i+1].replace('"""', '').strip()
            if not activity['name']:
                raise Exception('Activity {} should have a name'.format(activity['function_call']))

            # Description (multiline)
            activity['description'] = ''
            for k, line3 in enumerate(lines[i+3:]):
                if not line3.strip():
                    break
                activity['description'] += line3.strip()
            if not activity['description']:
                raise Exception('Activity {} should have a description'.format(activity['name']))

            # Parameters (multiline)
            activity['parameters'] = []
            activity['return'] = ''
            for l, line_param in enumerate(lines[i:]):
                if line_param.strip() == '@activity': # Till we reach next function
                    break
    
                if ':parameter' in line_param:
                    parameters = {}
                    parameters['name'] = line_param.split(':parameter')[-1].split(':')[0]
                    parameters['description'] = line_param.split(':',2)[-1]
                    
                    activity['parameters'].append(parameters)
                
                # Add return too
                if ':return:' in line_param:
                    activity['return'] = line_param.split(':',2)[-1]
            
            # Example (multiline)
            activity['example'] = ''
            for l, line4 in enumerate(lines[i:]):
                if line4.strip() == '@activity': # Till we reach next function
                    break

                if line4.strip() == ':Example:':
                    number_of_spaces = len(lines[i:][l+2]) - len(lines[i:][l+2].lstrip())
                    for m, line5 in enumerate(lines[i:][l+2:]):
                        if not line5.strip():
                            break
                        activity['example'] += line5[number_of_spaces:]
                    break
            
            # Add copy-paste example
            activity['snippet'] = ''
            for example_line in activity['example'].split('\n'):
                if example_line.strip():
                    if example_line[0] in ['>','#',' ']:
                        activity['snippet'] += example_line.replace('>>> ','') + '\n'
             
            if not activity['example']:
                raise Exception('Activity {} should have an example'.format(activity['name']))
                
            # Single lines
            for j, line2 in enumerate(lines[i:]):
                if line2.strip() == '@activity': # Till we reach next function
                    break

                if line2.strip() == 'Icon': # Icon
                    activity['icon'] = lines[i:][j+1].strip().replace('<i class="', '').replace('"></i>', '')

                if line2.strip() == 'Keywords': # Keywords
                    activity['keywords'] = lines[i:][j+1].strip().split(', ')

            category['activities'].append(activity)
            pprint(activity)
            if not activity.get('icon'):
                raise Exception('Activity {} should have an icon'.format(activity['name']))
        
import json
with open('activities.json', 'w') as f:
    json.dump(categories, f)

{'description': 'Generate random Fernet key. Fernet guarantees that a message '
                'encrypted using it cannot be manipulated or read without the '
                'key. Fernet is an implementation of symmetric (also known as '
                '“secret key”) authenticated cryptography',
 'example': '>>> # Generate a random key\n'
            '>>> generate_random_key()\n'
            "b'AYv6ZPVgnrUtHDbGZqAopRyAo9r0_UKrA2Rm3K_NjIo='\n",
 'function_call': 'generate_random_key()',
 'icon': 'las la-key',
 'keywords': ['random',
              'key',
              'fernet',
              'hash',
              'security',
              'cryptography',
              'password',
              'secure'],
 'name': 'Random key',
 'parameters': [],
 'return': ' Bytes-like object\n',
 'snippet': '# Generate a random key\ngenerate_random_key()\n'}
{'description': 'Encrypt text with (Fernet) key,',
 'example': '>>> # Generate a random key\n'
            '>>> key = generate_random_key()\n'
 

              'mouse automation',
              'click',
              'right click',
              'mouse button',
              'move mouse',
              'position',
              'pixel'],
 'name': 'Right click',
 'parameters': [{'description': ' ID of the element. To define an element and '
                                'attach an ID one can use the Automagica '
                                'recorder. The recorder uses vision to detect '
                                'an element and can be invoked with the '
                                'recorder() function.\n',
                 'name': ' id'},
                {'description': ' X-coördinate\n', 'name': ' x'},
                {'description': ' Y-coördinate\n', 'name': ' y'},
                {'description': ' Delay between right clicks in seconds, '
                                'standard value is 100 ms. \n',
                 'name': ' delay'}],
 'return': ' Right mouse click\n',
 'snippet': '# Click on a vision elemen

            "        'Column C': 'Data Row 2 for C',\n"
            '    }]\n'
            '>>> excel.insert_data_as_table(data)\n'
            '>>> # Get the table\n'
            ">>> excel.get_table('Table1')\n"
            "[['Column A', 'Column B', 'Column C'], ['Row 1 A Data', 'Row 1 B "
            "Data', 'Row 1 C Data'], ...]\n",
 'function_call': 'get_table(name)',
 'icon': 'las la-file-excel',
 'keywords': ['excel', 'worksheet names', 'tab names'],
 'name': 'Get table',
 'parameters': [{'description': ' List of table names\n', 'name': ' name'}],
 'return': '',
 'snippet': '# Open Excel\n'
            'excel = Excel()\n'
            '# Create a table (Table1)\n'
            'data = [\n'
            '    {\n'
            "        'Column A': 'Data Row 1 for A',\n"
            "        'Column B': 'Data Row 1 for B',\n"
            "        'Column C': 'Data Row 1 for C',\n"
            '    },\n'
            '    {\n'
            "        'Column A': 'Data Row 2 for A',\n"
    

{'description': 'Writes a list to a  text (.txt) file.Every element of the '
                'entered list is written on a new line of the text file.',
 'example': '>>> # Make a list to write\n'
            ">>> robot_names = ['WALL-E', 'Terminator', 'R2D2']\n"
            '>>> # Create a new text file\n'
            '>>> textfile = make_textfile()\n'
            '>>> write_list_to_file(robot_names, textfile)\n'
            '>>> # Open the file for illustration\n'
            '>>> open_file(textfile)\n',
 'function_call': 'write_list_to_file(list_to_write, file_path)',
 'icon': 'las la-list',
 'keywords': ['list', 'text', 'txt', 'list to file', 'write list', 'write'],
 'name': 'List to .txt',
 'parameters': [{'description': ' List to write to .txt file\n',
                 'name': ' list_to_write'},
                {'description': ' Path to the text-file. \n', 'name': ' path'}],
 'return': '',
 'snippet': '# Make a list to write\n'
            "robot_names = ['WALL-E', 'Terminator', 'R

In [3]:
%%capture cap --no-stderr

# Build activities.rst file

print('.. module:: automagica.activities')
print('')
print('Activities')
print('==========')
print('')
title = ''
for item in categories:
    print('\n')
    print(item['name'])
    underline = ''
    for char in range(0,len(item['name'])):
        underline = underline + '-'
    print(underline)
    
    for i in item['activities']:
        function = i['function_call']
        if function[0].isupper():
            class_name = i['function_call'].split('(')[0]
            class_name_rst = '.. autoclass:: ' + class_name +'\n' + '   :members:' 
            print(class_name_rst)
            break
        function_name = i['function_call'].split('(')[0]
        function_name_rst = '.. autofunction:: ' + function_name
        print(function_name_rst)

In [4]:
# Write to activities.rst file

with open(activities_rst_path, 'w') as f:
    f.write(cap.stdout)
    
print('Written to ' + activities_rst_path)

Written to C:\Users\TvT\repositories\automagica\docs\source\activities.rst


In [5]:
%%capture cap2 --no-stderr

# Build the readme.md

print('Process | Description')
print('------- | -----------')
title = ''
for item in categories:
    print('**' + item['name'] + '** | ‌‌ ')   
    for i in item['activities']:
        icon = i['icon']
        icon_name = str(i['icon'].split('la-')[-1])
        if icon_name not in ['html5', 'trello', 'salesforce', 'chrome', 'readme']:
            icon_name = icon_name + '-solid.svg'
        else:
            icon_name = icon_name + '.svg'
            
        icon_html = '<img src="https://github.com/OakwoodAI/automagica/blob/master/images/icons/' + icon_name + '" width="20">'
        name = i['name']
        docs_link = 'https://automagica.readthedocs.io/activities.html#automagica.activities.' + i['function_call'].split('(')[0]
        linked_name = '[' + str(name) + ']' + '(' + str(docs_link) + ')'
        description = i['description']
        line = icon_html + ' ' + linked_name + ' | ' + description
        print(line)
        
print('|<img width=150/>|  ‌‌|')

In [6]:
# Replace specific part in the readme.md with the new documentation

def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

with open(github_readme_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line_n,line in enumerate(lines):
    if "An overview of all official Automagica activities:" in line:
        starting_index = line_n + 2
    if "## Credits" in line:
        ending_index = line_n - 1 

new_readme = [] 
for printed_line in cap2.stdout.splitlines():
    new_readme.append(printed_line + '\n')

# Replace in readme
replace_sublist(lines, lines[starting_index:ending_index], new_readme)

with open(github_readme_path, "w", encoding='utf-8') as of:
    for line in lines:
        of.write(line)

